# Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install latest version of Huggingface and import necessary packages

In [2]:
!pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertModel
import json

Preparing the data for fine-tuning using OpenAI Playground

In [4]:
# Load training, test, and validation sets
PATH = '/content/drive/MyDrive/LLM_Assignment'
def load_data(PATH):
  train_data = pd.read_csv(PATH + '/training.csv')
  test_data = pd.read_csv(PATH + '/test.csv')
  val_data = pd.read_csv(PATH + '/validation.csv')
  return train_data, test_data, val_data

train_data, test_data, val_data = load_data(PATH)

In [ ]:
train_data

,Domain,area,keywords,Abstract
0,Medical,Hepatitis C,Feasibility study; hepatitis C risk behaviour...,Aims: This study aimed to develop and test the...
1,CS,Distributed computing,Agent Architecture; Mobile Agent; Agent Cloni...,Mobile agent technology is becoming more popul...
2,ECE,Control engineering,educational software tool; multivariable cont...,This paper presents an educational software to...
3,Psychology,False memories,judgment; metamemory; accuracy; eyewitness me...,"Different researchers have reported positive, ..."
4,Psychology,Leadership,Implementation support; Co-occurring disorder...,Background: Incorporating evidence-based integ...
5,CS,Bioinformatics,secondary metabolism; bioinformatics; genome ...,The soil-borne gram-positive bacteria Aneurini...
6,Psychology,Depression,Adolescence; Expressed emotion; Psychopatholo...,Objective: To investigate the association betw...
7,ECE,Electrical generator,CO2 capture; Efficiency; Exergy; LNG (liquefi...,The LNG (liquefied natural gas) regasification...
8,CS,Distributed computing,Quantum computing; Approximation algorithms; ...,We make use of a kind of distributed semi-quan...
9,Medical,Ankylosing Spondylitis,Biologic agents; Registry; Rheumatoid arthrit...,Despite improved quality of care for rheumatoi...


# Formatting Datasets

In [ ]:

def make_finetuning_data_gpt(dataset):
    abstracts = dataset['Abstract']
    domains = dataset['Domain']
    classified_data = []
    for abstract, domain in zip(abstracts, domains):
        message = {"messages": []}
        message["messages"].append({"role": "user", "content": abstract})
        message["messages"].append({"role": "assistant", "content": domain})
        classified_data.append(message)
    return classified_data

In [ ]:
processed_train_data = make_finetuning_data_gpt(train_data)
processed_val_data = make_finetuning_data_gpt(val_data)

In [ ]:
with open(PATH + '/training.jsonl', 'w') as f:
    for item in processed_train_data:
        json.dump(item, f)
        f.write('\n')  # Write a newline character to separate JSON objects

# Save validation data to JSONL
with open(PATH + '/validation.jsonl', 'w') as f:
    for item in processed_val_data:
        json.dump(item, f)
        f.write('\n')  # Write a newline character to separate JSON objects

Fine Tuning a Distilbert model on the training set


In [ ]:
training_set, test_set, validation_set = load_data(PATH)

In [ ]:
training_set['sample'] = 'training'
validation_set['sample'] = 'validation'
test_set['sample'] = 'test'

In [ ]:
num_classes = training_set['Domain'].nunique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
all_sets = pd.concat([training_set, validation_set, test_set], axis=0)

# Encode Domain column
label_encoder = LabelEncoder()
all_sets['Encoded_Domain'] = label_encoder.fit_transform(all_sets['Domain'])

# Split back into individual sets
training_set = all_sets[all_sets['sample'] == 'training']
validation_set = all_sets[all_sets['sample'] == 'validation']
test_set = all_sets[all_sets['sample'] == 'test']

In [ ]:
test_set

,Domain,area,keywords,Abstract,sample,Encoded_Domain
0,CS,Data structures,In-memory XPath processing; NESTOR; Set-based...,XML is a pervasive technology for representing...,test,0
1,Civil,Ambient Intelligence,Home energy management; persuasive interface;...,The integration of renewable energy sources in...,test,1
2,ECE,Electrical generator,non-standard electrical machine; generation o...,The original free-swinging piston engine with ...,test,2
3,Medical,Hepatitis C,complications; patient engagement; patient-ce...,Barriers to access and long-term complications...,test,4
4,ECE,Control engineering,force feedback haptic interface; virtual real...,This paper is to present a technological solut...,test,2
5,CS,Bioinformatics,Bioinformatics; genomics,Transposable elements (TEs) constitute the mos...,test,0
6,Medical,Weight Loss,Obesity; weight loss; moral work; body projec...,Cultural notions equating greater morality and...,test,4
7,Psychology,Leadership,Data-based decision making; school improvemen...,Although data-based decision making can lead t...,test,5
8,Psychology,Seasonal affective disorder,Ramelteon; sleep; agomelatine; depression; in...,Insomnia is common among elderly people and ne...,test,5
9,CS,Data structures,Succinct dynamic data structures; Succinct tr...,Cardinal trees (or tries of degree ) are a fun...,test,0


In [ ]:
def make_finetuning_data_distilbert(dataset):

    labels = dataset['Encoded_Domain'].values

    encoded = tokenizer.batch_encode_plus(
        dataset['Abstract'],
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt',
        truncation=True
    )

    # Create a tensor dataset
    return {
        'input_ids': encoded['input_ids'],
        'attention_mask': encoded['attention_mask'],
        'labels': torch.tensor(labels)
    }

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")

training_set = make_finetuning_data_distilbert(training_set)
validation_set = make_finetuning_data_distilbert(validation_set)
test_set = make_finetuning_data_distilbert(test_set)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Preparing the dataloader

In [ ]:
MAX_LENGTH = 128
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 16
BATCH_SIZE_TEST = 16

from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer

class FTDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):
        return {
            'input_ids': self.data['input_ids'][index],
            'attention_mask': self.data['attention_mask'][index],
            'labels': self.data['labels'][index]
        }

dataset_train = FTDataset(training_set)
dataloader_train = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE_TRAIN)
dataset_val = FTDataset(validation_set)
dataloader_val = DataLoader(dataset=dataset_val, batch_size=BATCH_SIZE_VAL)
dataset_test = FTDataset(test_set)
dataloader_test = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE_TEST)

In [ ]:
test_set

{'input_ids': tensor([[  101, 20950,  2003,  ...,  1998,  7926,   102],
         [  101,  1996,  8346,  ...,  1010,  1999,   102],
         [  101,  1996,  2434,  ...,     0,     0,     0],
         ...,
         [  101,  7863,  1024,  ...,  2099,  4442,   102],
         [  101,  1996,  5800,  ..., 13589, 13827,   102],
         [  101,  1037,  4338,  ...,  1010,  4525,   102]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([0, 1, 2, 4, 2, 0, 4, 5, 5, 0, 4, 0, 3, 2, 5, 4, 1, 6, 1, 1])}

Adding a trainable layer(s) on top of DistilBert

In [ ]:
class FTModel(nn.Module):
    def __init__(self, bert_model):
        super(FTModel, self).__init__()
        self.bert_model = bert_model
        self.dropout = nn.Dropout(0.1)  # Dropout layer for regularization
        self.classifier = nn.Linear(bert_model.config.hidden_size, num_classes)  # Add a linear layer for classification
        self.softmax = nn.Softmax(dim=1)  # Softmax activation for multi-class classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token embedding as the pooled output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        probabilities = self.softmax(logits)
        return logits, probabilities

# Create an instance of the FTModel
model = FTModel(bert_model)

In [ ]:
LEARNING_RATE = 2e-5
# Use cross-entropy loss
loss_fn = nn.CrossEntropyLoss()

# Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= LEARNING_RATE)

In [ ]:
# Freeze parameters of the pre-trained Distilbert model
for param in model.bert_model.parameters():
    param.requires_grad = False

Fine-tuning and evaluation functions

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
VAL_FREQUENCY = 10
PATH1 = '/content/drive/MyDrive/LLM_Assignment/Model.pt'


def evaluate(model, dataloader_val, loss_fn, device):
    model.eval()

    total_loss = 0
    accurate_predictions = 0
    total_samples = 0
    nb_eval_steps = 0

    for batch in dataloader_val:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
          logits = model(input_ids, attention_mask=attention_mask)[0]

        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        _, predictions = torch.max(logits, dim=1)
        accurate_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

        nb_eval_steps+=1

    avg_loss = total_loss / nb_eval_steps
    accuracy = accurate_predictions / total_samples
    print(f"Validation Loss: {avg_loss}, Accuracy: {accuracy}")
    return avg_loss

def finetune(epochs, model, loss_fn, optimizer, dataloader_train, dataloader_val, device):
    min_vloss = 10000
    patience_index = 0
    patience = 5

    for epoch in range(epochs):
        model.train()
        tr_loss = 0
        nb_tr_steps = 0
        loop = tqdm(enumerate(dataloader_train), leave=False, total=len(dataloader_train))

        for step, batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs[0], labels)
            loss.backward()
            optimizer.step()

            tr_loss += loss.item()
            nb_tr_steps += 1

        avg_train_loss = tr_loss / nb_tr_steps
        print("Train loss: {}".format(avg_train_loss))

        if epoch % VAL_FREQUENCY == 0:
            val_loss = evaluate(model, dataloader_val, loss_fn, device)
            if val_loss <= min_vloss:
                min_vloss = val_loss
                torch.save(model.state_dict(), PATH1)
                patience_index = 0
            else:
                patience_index += 1
            if patience_index == patience:
                break

    return model
model = finetune(200, model, loss_fn, optimizer, dataloader_train, dataloader_val, device)

Train loss: 2.130968928337097
Validation Loss: 2.268948554992676, Accuracy: 0.08


Train loss: 2.0794063210487366


Train loss: 2.1097753047943115


Train loss: 2.0849048495292664


Train loss: 2.093376874923706


...

Train loss: 1.667078673839569


Train loss: 1.684049665927887


Train loss: 1.6550106406211853


Train loss: 1.6790630221366882


Train loss: 1.6600894927978516


Test Set Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
def evaluate_on_test(model, test_set):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in test_set:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)[0]
            _, predicted = torch.max(outputs, dim=1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print(f"True Labels: {true_labels}")
    print(f"Predicted Labels: {predictions}")
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy


test_accuracy = evaluate_on_test(model, dataloader_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

True Labels: [0, 1, 2, 4, 2, 0, 4, 5, 5, 0, 4, 0, 3, 2, 5, 4, 1, 6, 1, 1]
Predicted Labels: [2, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6]
Test Accuracy: 20.00%


Fine-tune all parameters


In [5]:
training_set, test_set, validation_set = load_data(PATH)

In [6]:
training_set['sample'] = 'training'
validation_set['sample'] = 'validation'
test_set['sample'] = 'test'

In [7]:
num_classes = training_set['Domain'].nunique()

In [8]:
num_classes

7

In [9]:
from sklearn.preprocessing import LabelEncoder
all_sets = pd.concat([training_set, validation_set, test_set], axis=0)

label_encoder = LabelEncoder()
all_sets['Encoded_Domain'] = label_encoder.fit_transform(all_sets['Domain'])

training_set = all_sets[all_sets['sample'] == 'training']
validation_set = all_sets[all_sets['sample'] == 'validation']
test_set = all_sets[all_sets['sample'] == 'test']

In [10]:
def make_finetuning_data_distilbert(dataset):

    labels = dataset['Encoded_Domain'].values

    encoded = tokenizer.batch_encode_plus(
        dataset['Abstract'],
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt',
        truncation=True
    )

    return {
        'input_ids': encoded['input_ids'],
        'attention_mask': encoded['attention_mask'],
        'labels': torch.tensor(labels)
    }

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")

training_set = make_finetuning_data_distilbert(training_set)
validation_set = make_finetuning_data_distilbert(validation_set)
test_set = make_finetuning_data_distilbert(test_set)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
MAX_LENGTH = 128
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 16
BATCH_SIZE_TEST = 16

from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer

class FTDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):
        return {
            'input_ids': self.data['input_ids'][index],
            'attention_mask': self.data['attention_mask'][index],
            'labels': self.data['labels'][index]
        }

dataset_train = FTDataset(training_set)
dataloader_train = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE_TRAIN)
dataset_val = FTDataset(validation_set)
dataloader_val = DataLoader(dataset=dataset_val, batch_size=BATCH_SIZE_VAL)
dataset_test = FTDataset(test_set)
dataloader_test = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE_TEST)

In [12]:
bert_model_fullft = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [13]:
class FTModelFull(nn.Module):
    def __init__(self, bert_model, num_classes, second_num_classes):
        super(FTModelFull, self).__init__()
        self.bert_model = bert_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(bert_model.config.hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

        # Additional classifier
        self.second_classifier = nn.Linear(num_classes, second_num_classes)
        self.second_softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)

        logits = self.classifier(pooled_output)
        probabilities = self.softmax(logits)

        # Pass the output through the second classifier
        second_logits = self.second_classifier(probabilities)
        second_probabilities = self.second_softmax(second_logits)

        return second_logits, second_probabilities

In [14]:
# Initialize model but don't freeze Distilbert parameters
model_full_ft = FTModelFull(bert_model_fullft, num_classes, num_classes).to('cuda')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_full_ft = model_full_ft.to(device)

In [15]:
from transformers import AdamW

In [16]:
loss_fn = nn.CrossEntropyLoss()

# Choose parameters wisely!
learning_rate = 2e-5
adam_epsilon = 1e-8

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_full_ft.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.2},
    {'params': [p for n, p in model_full_ft.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
VAL_FREQUENCY = 10
PATH1 = '/content/drive/MyDrive/LLM_Assignment/Model2.pt'

# Function used to evaluate the model on
# the validation set
def evaluate(model, dataloader_val, loss_fn, device):
    model.eval()

    # Tracking variables
    total_loss = 0
    accurate_predictions = 0
    total_samples = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in dataloader_val:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
          logits = model(input_ids, attention_mask=attention_mask)[0]

        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        _, predictions = torch.max(logits, dim=1)
        accurate_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

        nb_eval_steps+=1

    avg_loss = total_loss / nb_eval_steps
    accuracy = accurate_predictions / total_samples
    print(f"Validation Loss: {avg_loss}, Accuracy: {accuracy}")
    return avg_loss

def finetune(epochs, model, loss_fn, optimizer, dataloader_train, dataloader_val, device):
    min_vloss = 10000
    patience_index = 0
    patience = 5

    for epoch in range(epochs):
        model.train()
        tr_loss = 0
        nb_tr_steps = 0
        loop = tqdm(enumerate(dataloader_train), leave=False, total=len(dataloader_train))

        for step, batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs[0], labels)
            loss.backward()
            optimizer.step()

            tr_loss += loss.item()
            nb_tr_steps += 1

        avg_train_loss = tr_loss / nb_tr_steps
        print("Train loss: {}".format(avg_train_loss))

        if epoch % VAL_FREQUENCY == 0:
            val_loss = evaluate(model, dataloader_val, loss_fn, device)
            if val_loss <= min_vloss:
                min_vloss = val_loss
                torch.save(model.state_dict(), PATH1)
                patience_index = 0
            else:
                patience_index += 1
            if patience_index == patience:
                break

    return model
model_full_ft = finetune(200, model_full_ft, loss_fn, optimizer, dataloader_train, dataloader_val, device)

Train loss: 1.9644247889518738
Validation Loss: 1.9453555047512054, Accuracy: 0.12


Train loss: 1.9509192109107971


Train loss: 1.9416365027427673


Train loss: 1.9280498027801514


Train loss: 1.917827069759369


...

Train loss: 1.7015182971954346


Train loss: 1.7015318274497986


Train loss: 1.701360046863556


Train loss: 1.7012701034545898


Train loss: 1.701372742652893
Validation Loss: 1.799689382314682, Accuracy: 0.54


Test Set Evaluation

In [18]:
from sklearn.metrics import accuracy_score
def evaluate_on_test(model, test_set):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in test_set:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)[0]
            _, predicted = torch.max(outputs, dim=1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print(f"True Labels: {true_labels}")
    print(f"Predicted Labels: {predictions}")
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy


test_accuracy = evaluate_on_test(model_full_ft, dataloader_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

True Labels: [0, 1, 2, 4, 2, 0, 4, 5, 5, 0, 4, 0, 3, 2, 5, 4, 1, 6, 1, 1]
Predicted Labels: [0, 0, 0, 4, 0, 6, 1, 1, 4, 0, 4, 0, 0, 0, 1, 4, 3, 6, 0, 3]
Test Accuracy: 35.00%
